# Satellite elevation-azimuth diagrams and PDOP
### Guillaume Witz, Science IT Support, Bern University

## Import packages

In [2]:
import os, re, glob
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
from bz2 import BZ2File as bzopen

import ipywidgets as ipw
from ipywidgets import interact, interactive, fixed, interact_manual

import aiub

## User parameters

In [3]:
#file location
address = 'http://ftp.aiub.unibe.ch/users/darnold/Python/'
#station coordinates file
coord_file = 'IGS14.CRD'
#station to observe
station_name = '14001M004'
#number of days to observe (starting from day == 0)
numdays = 3

## Elevation-azimuth
#### Data import and computation

In [4]:
#import station data
stations = aiub.import_stations(address,coord_file)

In [5]:
#select station and calculate ellipsoidal coordinates
curr_stat = stations[stations.statname == station_name].iloc[0]
rad_stat_coord = aiub.cartesian_to_ellipsoidal(curr_stat.x_pos, curr_stat.y_pos,curr_stat.z_pos)
curr_stat['rad_stat_coord'] = rad_stat_coord

In [9]:
temp_pd

,index,year,month,day,hour,minute,satellite,name,c1,c2,...,c4,time_stamp,key,datetime,Xgeo,Ygeo,Zgeo,a,e,curr_stat
0,0,2019,1,1,0,0,GPS,PG01,1.514994e+06,-1.610703e+07,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,4.173004e+06,-1.541336e+07,-2.690926e+07,2.191893,-0.892468,40421M002
1,1,2019,1,1,0,0,GPS,PG02,-2.055908e+07,1.537251e+07,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,-1.790107e+07,1.606619e+07,1.234466e+06,-2.089261,0.286726,40421M002
2,2,2019,1,1,0,0,GPS,PG03,-8.396509e+06,-2.005298e+07,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,-5.738499e+06,-1.935931e+07,-2.108140e+07,2.389748,-0.521933,40421M002
3,3,2019,1,1,0,0,GPS,PG04,1.307618e+07,-1.395804e+07,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,1.573419e+07,-1.326436e+07,1.219925e+07,0.811158,0.249907,40421M002
4,4,2019,1,1,0,0,GPS,PG05,-1.152675e+07,1.053681e+07,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,-8.868744e+06,1.123049e+07,1.563973e+07,-1.454683,0.898937,40421M002
5,5,2019,1,1,0,0,GPS,PG06,-2.519528e+07,6.146615e+06,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,-2.253727e+07,6.840290e+06,-1.148904e+07,-2.651404,-0.066881,40421M002
6,6,2019,1,1,0,0,GPS,PG07,-1.437622e+07,-9.834349e+06,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,-1.171821e+07,-9.140674e+06,1.467751e+07,2.367496,1.155945,40421M002
7,7,2019,1,1,0,0,GPS,PG08,6.182801e+06,-2.535496e+07,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,8.840811e+06,-2.466129e+07,-1.303580e+06,1.511793,-0.083076,40421M002
8,8,2019,1,1,0,0,GPS,PG09,-8.966383e+06,-1.932678e+07,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,-6.308372e+06,-1.863311e+07,1.007137e+07,1.890192,0.670789,40421M002
9,9,2019,1,1,0,0,GPS,PG10,2.303614e+07,9.270331e+06,...,999999.999999,1.546297e+09,1,2019-01-01 00:00:00,2.569415e+07,9.964006e+06,-1.537786e+07,-0.172283,-0.948665,40421M002


In [6]:
#import satellite data
temp_pd = pd.concat([aiub.import_RM_file(address = address, day = i) for i in range(numdays)]).reset_index()
#reanme satellite types
sat_dict = {'G':'GPS','R':'GLONASS','E':'Galileo','C':'BeiDou','J':'QZSS'}
temp_pd['satellite'] = temp_pd.satellite.apply(lambda x : sat_dict[x])

In [7]:
#caluclate elevation and azimuth for all satellites for given station
temp_pd = aiub.el_al_single_station_fast(temp_pd, curr_stat)
temp_pd['curr_stat'] = curr_stat.statname

### Plotting

In [8]:
aiub.interactive_rad(temp_pd, stations)

Output()

## PDOP

### User input

In [8]:
#how many days to include for calculation
numdays = 10

### Data import and calculation

In [10]:
#import data and compute elevation-azimuth
temp_pd = pd.concat([aiub.import_RM_file(address = address, day = i) for i in range(numdays)]).reset_index()
temp_pd = aiub.el_al_single_station_fast(temp_pd, curr_stat)


In [11]:
#remove elevation < 5°
temp_pd = temp_pd[temp_pd.e >2*np.pi*5/360]


In [12]:
#calculate norm of satellite-station vector Xgeo
temp_pd['Xgeo_norm'] = temp_pd.apply(lambda row: np.linalg.norm([row['Xgeo'],row['Ygeo'],row['Zgeo']]),axis = 1)


In [13]:
#group data by chuncks of 30min
time_grouped = temp_pd.groupby(pd.Grouper(key='datetime', freq='30min', axis=1))


In [14]:
#calculate pdop for each group
pdop = dict((key, []) for key in temp_pd.satellite.unique())

for name, df0 in time_grouped:
    sat_grouped = df0.groupby('satellite')
    for name2, df in sat_grouped:
        A_mat = np.stack([-df['Xgeo']/df['Xgeo_norm'],-df['Ygeo']/df['Xgeo_norm'],
              -df['Zgeo']/df['Xgeo_norm'],np.ones(len(df))],axis = 1)

        try:
            inv_mat = np.linalg.inv(np.matmul(A_mat.T,A_mat))
        except:
            False

        pdop[name2].append(np.sqrt(inv_mat[0,0]+inv_mat[1,1]+inv_mat[2,2]))

/Users/gw18g940/miniconda3/envs/astronomy/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app


In [15]:
#calculate power spectrum and set the right (?) scale 
powerspec = {}
for x in pdop.keys():
    powerspec[x] = scipy.signal.periodogram(pdop[x],fs = 2/3600)#, scaling='spectrum')

### Plotting

In [16]:
#define interactive plotting function 
def plot_pdop(sat_type, pdop):
    fig, ax = plt.subplots(2,1)
    for s in sat_type:
        ax[0].plot(np.arange(len(pdop[s]))/(24*2),pdop[s], label = s)
        
        ax[1].plot(powerspec[s][0]*3600*24,powerspec[s][1], label = s)
    ax[0].legend()
    ax[1].set_xlim((0,5))
    plt.show()

In [17]:
ipw.interactive(plot_pdop, sat_type = ipw.SelectMultiple(options =temp_pd.satellite.unique()),
                pdop = ipw.fixed(pdop))

interactive(children=(SelectMultiple(description='sat_type', options=('G', 'R', 'E', 'C'), value=()), Output()…